In [1]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics

from siuba import *
import pandas as pd
import geopandas as gpd

import datetime as dt

import importlib
import rt_analysis as rt
import rt_filter_map_plot

from IPython.display import display, Markdown, Latex, HTML
import json

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

In [2]:
importlib.reload(rt_filter_map_plot)

<module 'rt_filter_map_plot' from '/home/jovyan/data-analyses/rt_delay/rt_filter_map_plot.py'>

In [3]:
# itp_id = 4 ## AC Transit
itp_id = 159 ## Lake Transit

In [4]:
import numpy as np

In [5]:
date_range = np.arange('2022-04-30', '2022-05-09', dtype='datetime64[D]')

In [6]:
date_range

array(['2022-04-30', '2022-05-01', '2022-05-02', '2022-05-03',
       '2022-05-04', '2022-05-05', '2022-05-06', '2022-05-07',
       '2022-05-08'], dtype='datetime64[D]')

# Generate analysis data for all dates

In [7]:
pbar = tqdm()

0it [00:00, ?it/s]

In [8]:
# for date in date_range:
#     date = date.astype(dt.date)
#     try:
#         rt_analysis = rt.OperatorDayAnalysis(itp_id, date, pbar)
#         rt_analysis.export_views_gcs()
#         print(f'complete for date: {date}')
#     except Exception as e:
#         print(e)
#         print(f'failed for date: {date}')
#         continue

In [9]:
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date)
        print(f'complete for date: {date}')
    except:
        print(f'failed for date: {date}')
        continue

failed for date: 2022-04-30
failed for date: 2022-05-01
found parquet
complete for date: 2022-05-02
found parquet
complete for date: 2022-05-03
found parquet
complete for date: 2022-05-04
found parquet
complete for date: 2022-05-05
found parquet
complete for date: 2022-05-06
failed for date: 2022-05-07
failed for date: 2022-05-08


In [10]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, dt.date(2022, 5, 4))

found parquet


In [11]:
# corridor = gpd.read_file('./CMCP_StudyAreaBoundary.geojson') ## alameda
corridor = gpd.read_file('./corridors_sccp_lpp/lakeport_map.geojson') ## lakeport

In [12]:
rt_day.add_corridor(corridor)

In [13]:
mappable_stops = (rt_day.stop_delay_view
          >> distinct(_.shape_id, _.stop_sequence, _keep_all=True)
          >> filter(_.corridor)
          >> select(_.stop_id, _.geometry, _.stop_sequence)
         )

In [14]:
# quick corridor map
rt_day.quick_map_corridor()

# Algorithm

* Clip stops to corridor
* Get first and last stops in corridor for each shape_id, then find 1 stop before and 1 stop after for each
* Filter stop_delay_view based on those stops
* Assert that more than 50% of filtered stops within corridor, if not then drop shape (remove shapes starting/ending in corridor but mostly running outside)

# Report:

* Sum of median trip delays
* Median segment speed
* Delay compared to speed standards: 16mph (approx. 75th %ile for medium size operator by 2010 NTD)

# Corridor-based metrics for SCCP

* intent is to eventually parameterize this and output formatted report, also methodology may change

In [15]:
pbar = tqdm()

0it [00:00, ?it/s]

In [19]:
rt_day.corridor_metrics()

{'schedule_metric_minutes': 19.233333333333334,
 'speed_metric_minutes': -583.3004503960505}

In [22]:
schedule_metrics = []
speed_metrics = []
for date in date_range:
    date = date.astype(dt.date)
    try:
        rt_day = rt_filter_map_plot.from_gcs(itp_id, date, pbar)
        rt_day.add_corridor(corridor)
        _m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)
        metrics = rt_day.corridor_metrics()
        schedule_metrics += [metrics['schedule_metric_minutes']]
        speed_metrics += [metrics['speed_metric_minutes']]
        print(f'complete for date: {date}')
    except Exception as e:
        print(f'failed for date: {date}')
        print(e)
        continue

failed for date: 2022-04-30
calitp-analytics-data/data-analyses/rt_delay/rt_trips/159_04_30.parquet
failed for date: 2022-05-01
calitp-analytics-data/data-analyses/rt_delay/rt_trips/159_05_01.parquet
found parquet
complete for date: 2022-05-02
found parquet
complete for date: 2022-05-03
found parquet
complete for date: 2022-05-04
found parquet
complete for date: 2022-05-05
found parquet
complete for date: 2022-05-06
failed for date: 2022-05-07
calitp-analytics-data/data-analyses/rt_delay/rt_trips/159_05_07.parquet
failed for date: 2022-05-08
calitp-analytics-data/data-analyses/rt_delay/rt_trips/159_05_08.parquet


In [23]:
np.round(np.mean(schedule_metrics), 0)

10.0

In [24]:
np.round(np.mean(speed_metrics), 0)

-463.0

In [29]:
rt_day = rt_filter_map_plot.from_gcs(itp_id, dt.date(2022, 5, 4), pbar)
rt_day.add_corridor(corridor)
_m = rt_filter_map_plot.RtFilterMapper.segment_speed_map(rt_day, corridor=True)

found parquet


In [30]:
_m.save('lake_map.html')

In [27]:
rt_day.stop_delay_view >> filter(_.corridor) >> distinct(_.route_short_name)

,route_short_name
0,11
1,4
2,3
3,10
